In [8]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import math
from numpy import random
from networkx import linalg
import itertools
import pandas as pd
import multiprocessing as mp
import time
import csv

%matplotlib inline

In [9]:
mouse_brain_vasc_network = pd.read_pickle('../data/mouse_brain_vasc_network_2019.pkl')
mouse_brain_vasc_network.head(5)

,V1,VX1,VY1,VZ1,V2,VX2,VY2,VZ2,dt,length
0,1.0,2733.0,5686.0,6.0,5.0,2742.000000,5683.000000,8.000000,2.923270,135.594335
1,1.0,2733.0,5686.0,6.0,6.0,2720.666667,5688.000000,8.333333,1.878005,2.000000
2,1.0,2733.0,5686.0,6.0,2.0,2739.000000,5692.000000,6.000000,2.273637,332.997078
3,5.0,2742.0,5683.0,8.0,17.0,2761.000000,5681.000000,12.000000,2.168766,22.145535
4,3.0,2671.0,5688.0,7.0,57.0,2649.666667,5670.666667,20.666667,1.887354,22.973963


In [4]:
global inity,maxx,minx,maxz,minz,maxy,miny,mouse_brain_vasc_network
maxx = max([mouse_brain_vasc_network['VX1'].max(), mouse_brain_vasc_network['VX2'].max()])
minx = min([mouse_brain_vasc_network['VX1'].min(), mouse_brain_vasc_network['VX2'].min()])
maxy = max([mouse_brain_vasc_network['VY1'].max(), mouse_brain_vasc_network['VY2'].max()])
miny = min([mouse_brain_vasc_network['VY1'].min(), mouse_brain_vasc_network['VY2'].min()])
maxz = max([mouse_brain_vasc_network['VZ1'].max(), mouse_brain_vasc_network['VZ2'].max()])
minz = min([mouse_brain_vasc_network['VZ1'].min(), mouse_brain_vasc_network['VZ2'].min()])

print(maxy,miny)
print(maxz,minz)
print(maxx,minx)

10415.0 225.0
13985.0 6.0
7243.0 433.0


In [10]:
def extract_region(lb,delta,ax):
    d = {'X':['VX1','VX2'],'Y':['VY1','VY2'],'Z':['VZ1','VZ2']}     
    indices1 = list(mouse_brain_vasc_network[(mouse_brain_vasc_network[d[ax][0]] <= lb+delta) \
                                         & (mouse_brain_vasc_network[d[ax][0]] >= lb)].index)
    indices2 = list(mouse_brain_vasc_network[(mouse_brain_vasc_network[d[ax][1]] <= lb+delta) \
                                         & (mouse_brain_vasc_network[d[ax][1]] >= lb)].index)
    if indices1 and indices2:
        return np.intersect1d(np.array(indices1),np.array(indices2)).flatten()
    else:
        return np.array([])

In [11]:
def extractgraph_indices(initx,initz):
    indicesx = extract_region(initx,100,'X')
    indicesz = extract_region(initz,100,'Z')
    if (indicesx.size>0) and (indicesz.size>0):
        #print(f'non-empty-{0}')
        indices = np.intersect1d(indicesx,indicesz).flatten()
        indicesy = extract_region(inity,100,'Y')
        if (indices.size > 0) and (indicesy.size > 0):
            #print(f'non-empty-{1}')
            indices = np.intersect1d(indices,indicesy).flatten()
            if indices.size > 0:
                #print(f'non-empty-{2}')
                G = nx.Graph()
                for k in indices:
                    G.add_edge(mouse_brain_vasc_network['V1'][k], mouse_brain_vasc_network['V2'][k],\
                               dt = mouse_brain_vasc_network['dt'][k],\
                               length = mouse_brain_vasc_network['length'][k])

                el_density = 0.0
                for e in list(G.edges.data()):
                    el_density = el_density + e[2]['length']
                el_density = el_density/100**3
                
                node_density = len(G.nodes())/100**3
                H = nx.Graph()
                for e in list(G.edges.data()):
                    H.add_edge(e[0], e[1], weight = e[2]['dt'])

                return [H, initx, inity, initz, 100, node_density, el_density]
            else:
                return None
        else:
            return None

In [ ]:
M = np.median(np.arange(miny,maxy,100))
print(M)
for k in np.arange(4300,M,100):
    
    inity = np.median(np.arange(miny,maxy,100))-k
    print(inity)
                    
    pool = mp.Pool(12)
    results = [pool.apply_async(extractgraph_cylindricalregions, args=item)\
               for item in list(itertools.product(np.arange(minx,maxx,100),np.arange(minz,maxz,100)))]
    pool.close()
    pool.join()
    
    output = [p.get() for p in results]
    not_none = [item for item in output if item != None]
    
    s = '../data/box_graph_data_2019_100/network_data_box_sampling_nooverlaps_m_100_' f'{int(k)}' + '.csv'
    print(s)
    with open(s, 'a') as fh:
        writer = csv.writer(fh)
        for g in not_none:
            writer.writerow([0,0,0,0,0,0])
            writer.writerow([g[1],g[2],g[3],g[4],g[5],g[6]])
            for t in list(g[0].edges.data()):
                writer.writerow([t[0],t[1],t[2]['weight']])

975.0
